In [1]:
from bs4 import BeautifulSoup
import requests
import math
import pandas as pd

In [2]:
url = 'https://ar.wikipedia.org/wiki/%D8%A3%D8%AD%D9%8A%D8%A7%D8%A1_%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6' 
response = requests.get(url)
response.status_code
page = response.text
soup = BeautifulSoup(page, "lxml")

In [3]:
districts = ["طويق", "الحائر", "الاندلس", "الشرفية", "الشفا", "البيان", "الصالحية", "غبيرة", "العيينة", "ام سليم", "الشرق"]
table = soup.find(class_="wikitable")
rows = soup.find_all("tr")
dist = []
for r in rows:
    tds = r.find_all("td")
    if len(tds) >= 2:
        districts.append(tds[1].text)

In [7]:
fixed_districts = []
for d in districts:
    fixed = d.replace("حي ", "")
    fixed = fixed.replace("الغربي ", "")
    fixed = fixed.replace("الشرقي ", "")
    fixed = fixed.replace("الغربية ", "")
    fixed = fixed.replace("الشرقية ", "")
    fixed = fixed.replace("الشرقية ", "")
    fixed = fixed.replace("الشرقية ", "")
    fixed = fixed.replace("\n", "")
    
    if(len(fixed) > 1):
        valid = True
        for character in fixed:
            if character.isdigit():
                valid = False
                break

        if valid:
            fixed_districts.append(fixed)

In [8]:
print(list(set(fixed_districts)))

['قرطبة', 'النسيم الشرقي', 'الشرفية', 'العزيزية', 'بنبان', 'المنصورية', 'جبرة', 'السويدي', 'هجرة وادي لبن', 'صياح', 'الجزيرة', 'الحاير', 'البيان', 'العليا', 'الإسكان', 'النرجس', 'الوسيطاء', 'النموذجية', 'المؤتمرات', 'الصفا', 'الرمال', 'النظيم', 'المرقب', 'الريان', 'الوادي', 'الشرقية', 'المصانع', 'الدفاع', 'الديرة', 'النهضة', 'اليمامة', 'عتيقة', 'البطيحا', 'أشبيلية', 'نمار', 'الصحافة', 'الهدا', 'النسيم الغربي', 'الصناعية', 'صلاح الدين', 'البرية', 'الفلاح', 'القادسية', 'المعذر', 'العيينة', 'سلام', 'المونسية', 'الرحمانية', 'المعيزلية', 'الحائر', 'أم الحمام الغربي', 'الندى', 'المغرزات', 'الشرق', 'العقيق', 'الرفيعة', 'المصيف', 'الملك عبد العزيز', 'الملك عبد الله', 'المصفاة', 'الفوطة', 'الغنامية', 'المنار', 'جرير', 'العمل', 'سلطانة', 'الربوة', 'الحمراء', 'خشم العان', 'القرى', 'طويق', 'عرقة', 'الاندلس', 'العريجاء الغربية', 'الشفاء', 'عريض', 'شبرا', 'الوزارات', 'أحد', 'بدر', 'أم الحمام الشرقي', 'ثليم', 'أم سليم', 'الملز', 'الرائد', 'العريجاء', 'الأندلس', 'الفاروق', 'الفيحاء', 'النفل', 'النخيل'

In [9]:
def prepare_data(houses_list, soup, districts):
    houses=soup.find_all(class_='titleAndDetails')
    headers = ["PRICE", "BED", "BATH", "SIZE", "DISTRICT"]

    for house in houses:
        price = int(house.find(class_="price").text.replace("ريال", "").replace(",", "").strip())
        bed = int(house.find(class_="bed").text.strip())
        bath = int(house.find(class_="bath").text.strip())
        size = int(house.find(class_="size").text.replace("م²", "").strip())
        address = house.find(class_="listTitle").text.strip()

        house_dist = None
        for district in districts:
            if district in address:
                house_dist = district.strip()
                break
            
        house_dict = dict(zip(headers, [price, bed, bath, size, house_dist]))
        houses_list.append(house_dict)

In [10]:
houses_list = []
i = 1
while i < 50:
    url = f'https://sa.aqar.fm/%D8%A8%D9%8A%D8%AA-%D9%84%D9%84%D8%A8%D9%8A%D8%B9/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/{i}' 
    response = requests.get(url)
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        prepare_data(houses_list, soup, fixed_districts)
        i += 1
    else:
        break

In [11]:
houses_df = pd.DataFrame(houses_list)
houses_df

,PRICE,BED,BATH,SIZE,DISTRICT
0,100000,4,1,137,ام سليم
1,200000,5,4,175,اليمامة
2,1200000,5,3,552,الشرق
3,1050000,4,5,321,الرمال
4,300000,5,4,93,منفوحة
...,...,...,...,...,...
965,850000,4,5,200,ظهرة لبن
966,5000000,5,5,2200,الناصرية
967,1000000,4,3,315,الجنادرية
968,1100000,4,5,360,طويق


In [14]:
houses_df.DISTRICT.value_counts().reset_index()

,index,DISTRICT
0,منفوحة,13
1,الندوة,13
2,البديعة,12
3,طويق,11
4,اليمامة,9
5,الجرادية,7
6,بدر,7
7,الدار البيضاء,7
8,الصالحية,5
9,الرمال,5


In [ ]:
books